# Проверочная работа по теме "Критерии Стьюдента"

In [67]:
import numpy as np
import pandas as pd

from statsmodels.stats import weightstats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import scipy

In [69]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Задание 1

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

## Решение

### Дано
Первая группа: здоровые молодые женщины.   
$\mu = 9.5$  
$\sigma = 0.4$

Вторая группа: беременные женщины Гватемалы.    
$n = 160$  
$\bar x = 9.57$ 

### Гипотезы:

$H_0\colon$ Средний уровень кальция в этой популяции равен 9.5 ($\mu_{беременных} = \mu = 9.5$)    
     
$H_1\colon$ Средний уровень кальция в этой популяции отличается от 9.5 ($\mu_{беременных} \neq \mu_ \neq 9.5$)

Используем z-критерий, так как нам известна сигма.   
Статистика будет выглядеть следующим образом:   
$Z = \frac{\bar x - \mu_0}{\sigma / \sqrt{n}}$

In [1]:
mu = 9.5
sigma = 0.4

pregnant_mean = 9.57
n = 160

In [4]:
z = (pregnant_mean - mu) / (sigma / np.sqrt(160))
z

2.213594362117875

In [25]:
p = 2 * (1 - weightstats.stats.norm.cdf(np.abs(z)))
round(p, 4)

0.0269

На уровне значимости $\alpha$ = 95% мы можем отвергнуть нулевую гипотезу, т.е. средний уровень кальция в популяции здоровых беременных женщин Гватемалы отличается от 9.5.

## Задание 2

Имеются данные о стоимости и размерах 53940 бриллиантов (diamonds.txt).   

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

1. линейную регрессию с помощью LinearRegression без параметров

2.  случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

## Решение

### Смотрим данные

In [27]:
diamonds = pd.read_csv('./data/diamonds.txt', sep='\t')

In [29]:
diamonds.head(3)

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31


In [30]:
diamonds.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [31]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   depth   53940 non-null  float64
 2   table   53940 non-null  float64
 3   price   53940 non-null  int64  
 4   x       53940 non-null  float64
 5   y       53940 non-null  float64
 6   z       53940 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


### Train-Test Split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(diamonds.drop(['price'], axis=1), diamonds['price'], test_size=0.25, random_state=1)

In [41]:
X_train.shape

(40455, 6)

In [42]:
X_test.shape

(13485, 6)

### Линейная регрессия с помощью LinearRegression без параметров

In [47]:
linear_model = LinearRegression().fit(X_train, y_train)

In [48]:
linear_prediction = linear_model.predict(X_test)

In [51]:
linear_prediction[:5]

array([ 778.42235621, 6930.46053685, 2520.12128307,  529.83435479,
       7077.15270117])

In [57]:
# модули отклонений предсказаний от истинных цен
linear_err = np.abs(linear_prediction - y_test)

In [64]:
linear_err.mean()

890.3764004285589

### Cлучайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1

In [54]:
random_forest_model = RandomForestRegressor(n_estimators=10, random_state=1).fit(X_train, y_train)

In [55]:
random_forest_prediction = random_forest_model.predict(X_test)

In [56]:
random_forest_prediction[:5]

array([ 858.55, 7205.6 , 2108.8 ,  579.  , 8339.5 ])

In [65]:
# модули отклонений предсказаний от истинных цен
random_forest_err = np.abs(random_forest_prediction - y_test)

In [66]:
random_forest_err.mean()

802.9205172724141

### Сравним качесво моделей 

Имеем две независимые выборки, нужно сравнить среднее этих выборок.   
Попробуем использовать **двухвыборочный критерий Стьюдента**.  

Нам неизвестны сигмы, будем использовать t-критерий.  
Проблема Боренца-Фишера: аппроксимация будет достаточно точна, так как размеры двух сравниваемых выборок одинаковые.

#### Гипотезы:  
$H_0\colon \mu_1 = \mu_2$ Качество двух моделей не отличается   

$H_1\colon \mu_1 \neq \mu_2$ Качество двух моделей отличается

In [77]:
scipy.stats.ttest_ind(linear_err, random_forest_err, equal_var=False)

Ttest_indResult(statistic=6.2054184467870765, pvalue=5.534495566292084e-10)

Получили очень маленькое значение p-value, следовательно, мы можем отвергнуть нулевую гипотезу и принять альтернативную, т.е. качество двух моделей отличается.

## Задание 3

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [78]:
cm = weightstats.CompareMeans(weightstats.DescrStatsW(linear_err), weightstats.DescrStatsW(random_forest_err))
print(f"Conf interval: {cm.tconfint_diff(alpha=0.05)}")

Conf interval: (59.831948333829374, 115.07981797846968)
